In [ ]:
import os
import sys
import rasterio
import numpy as np
import xarray as xr
from scipy.interpolate import RectBivariateSpline
from rasterio.transform import Affine
import matplotlib.pyplot as plt

#wqsat_format
from wqsat_format import utils, preprocessing
from wqsat_format import s3_reader

In [2]:
def read_sentinel3_bands(input_folder):
    """
    Reads all Sentinel-3 bands from a .SEN3 folder (NetCDF format) and returns them as a dictionary.
    
    Parameters:
    ----------
    input_folder : str
        Path to the .SEN3 folder containing Sentinel-3 NetCDF files.

    Returns:
    -------
    arr_bands : dict
        Dictionary with band names as keys and NumPy arrays as values.
    metadata : dict
        Dictionary with general metadata of the image.
    """
    arr_bands = {}
    metadata = {}

    # List all NetCDF files inside the .SEN3 folder
    netcdf_files = [f for f in os.listdir(input_folder) if f.endswith("radiance.nc")]

    # List of Sentinel-3 bands
    bands = ['Oa01', 'Oa02', 'Oa03', 'Oa04', 'Oa05', 'Oa06', 'Oa07', 'Oa08', 'Oa09', 'Oa10', 
    'Oa11', 'Oa12', 'Oa13', 'Oa14', 'Oa15', 'Oa16', 'Oa17', 'Oa18', 'Oa19', 'Oa20', 'Oa21']

    for i, file in enumerate(netcdf_files):
        try:
            file_path = os.path.join(input_folder, file)
            src = rasterio.open(file_path)
            arr_bands[bands[i]] = src.read(1)

        except Exception as e:
            print(f"Error reading {file_path}: {e}")

    # Extract metadata
    metadata = src.profile
    metadata.update({
        "count": len(arr_bands),  # Number of bands successfully read
        "bands": list(arr_bands.keys()),  # List of bands read
        "bounds": src.bounds, # Bounding box of the image
    })

    return arr_bands, metadata

In [4]:
def s3_sun_angle(tile_path):


    file_path = os.path.join(tile_path, 'tie_geometries.nc')
    ds = xr.open_dataset(file_path)

    # Extraer SZA y SAA
    sza = ds["SZA"].values  # (4091, 77)
    saa = ds["SAA"].values  # (4091, 77)

    # Crear mallas de coordenadas originales
    tie_x = np.linspace(0, 4864, 77)  # Puntos de atadura en X
    tie_y = np.linspace(0, 4090, 4091)  # Puntos de atadura en Y

    # Crear mallas de destino a resolución completa
    full_x = np.linspace(0, 4864, 4865)  # Resolución completa en X
    full_y = np.linspace(0, 4090, 4091)  # Resolución completa en Y

    # Interpolación bilineal
    sza_interp = RectBivariateSpline(tie_y, tie_x, sza)(full_y, full_x)
    saa_interp = RectBivariateSpline(tie_y, tie_x, saa)(full_y, full_x)

    sun_angle = {'sza': sza_interp, 'saa': saa_interp}
    return sun_angle

In [ ]:
class AtcorSentinel3:
    def __init__(self, arr_bands, sun_angle, scale_factor):
        """
        Atmospheric Correction Class for Sentinel-3 OLCI L1 products.

        Parameters:
        - arr_bands (dict): Dictionary containing image bands organized by band names.
          Example structure:
          {
              'Oa01': arr_o1, 'Oa02': arr_o2, 'Oa03': arr_o3, ...
          }
        - sun_zenith (dict): Dictionary containing solar zenith angles (SZA) for each band (same dimensions as the bands).
          Example structure:
          {
              'Oa01': arr_sza_o1, 'Oa02': arr_sza_o2, ...
          }
        - sun_azimuth (dict): Dictionary containing solar azimuth angles (SAA) for each band (same dimensions as the bands).
          Example structure:
          {
              'Oa01': arr_saa_o1, 'Oa02': arr_saa_o2, ...
          }
        - scale_factor (int, optional): Scaling factor for reflectance values (default: 10000).
        """
        
        self.arr_bands = arr_bands  # Dictionary of band arrays
        self.sun_zenith = sun_angle.get('sza')  # Dictionary of solar zenith angles (same dimensions as the bands)
        self.sun_azimuth = sun_angle.get('saa')  # Dictionary of solar azimuth angles (same dimensions as the bands)
        self.scale_factor = scale_factor
    
    def apply_scale_factor(self):
        """Normalize the reflectance values by dividing by the scale factor."""
        for band, arr in self.arr_bands.items():
            self.arr_bands[band] = arr / self.scale_factor
        return self.arr_bands
    
    def sun_angle_correction(self):
        """Apply solar angle correction using both the solar zenith angle (SZA) and solar azimuth angle (SAA)."""
        # Ensure that SZA and SAA arrays are available
        if self.sun_zenith is None or self.sun_azimuth is None:
            raise ValueError("Sun zenith or azimuth angle arrays are missing.")
        
        # Apply the solar zenith correction for each band
        for band, arr in self.arr_bands.items():
            # Apply the correction using the solar zenith angle (SZA)
            sza_rad = np.radians(self.sun_zenith)  # Convert SZA to radians
            self.arr_bands[band] = arr / np.cos(sza_rad)  # Apply the correction
        
        return self.arr_bands
    
    def dark_object_subtraction(self):
        """Apply DOS (Dark Object Subtraction) correction by subtracting the minimum value in each band."""
        for band, arr in self.arr_bands.items():
            dark_object_value = np.percentile(arr, 1)  # Approximate dark object (1st percentile)
            arr = arr - dark_object_value
            self.arr_bands[band] = np.clip(arr, 0, None)  # Ensure no negatives
        return self.arr_bands
    
    def apply_all_corrections(self):
        """Apply all corrections (scale factor, solar angle, and DOS) in sequence."""
        self.apply_scale_factor()
        self.sun_angle_correction()
        self.dark_object_subtraction()
        return self.arr_bands

In [6]:
def export_sentinel3_to_geotiff(data_bands, metadata, output_dir):
    """
    Exports Sentinel-3 data bands to GeoTIFF format.

    Parameters:
        data_bands (dict): A dictionary containing data for each band of Sentinel-3 (e.g. 'Oa01', 'Oa02', ...).
        metadata (dict): Metadata dictionary with information such as image dimensions, CRS, etc.
        output_dir (str): The directory where the exported files will be saved.

    Returns:
        None
    """
    try:
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Sort the bands by their names
        list_bands = sorted(data_bands.keys())  # Ensure band order consistency
        stacked_bands = np.stack([data_bands[band] for band in list_bands], axis=0)

        # Get metadata
        meta = metadata.copy()
        meta.update({
            "count": stacked_bands.shape[0],  # Number of bands
            "dtype": stacked_bands.dtype,
            "bands": list_bands
        })

        # Create output file name
        output_file = os.path.join(output_dir, "sentinel3_bands.tif")

        # Write to GeoTIFF
        with rasterio.open(output_file, 'w', driver='GTiff',
                           height=stacked_bands.shape[1], width=stacked_bands.shape[2],
                           count=stacked_bands.shape[0], dtype=stacked_bands.dtype,
                           crs=meta["crs"], transform=meta["transform"]) as dst:
            for band_idx in range(stacked_bands.shape[0]):
                dst.write(stacked_bands[band_idx, :, :], band_idx + 1)

        # Save metadata to a separate text file
        metadata_file = os.path.join(output_dir, "metadata_sentinel3.txt")
        with open(metadata_file, 'w') as f:
            f.write("Metadata for Sentinel-3 Bands:\n")
            for key, value in meta.items():
                f.write(f"{key}: {value}\n")

    except Exception as e:
        raise RuntimeError(f"Error exporting Sentinel-3 bands: {e}")

In [ ]:
input_path = '***'
tiles = os.listdir(input_path)
pending = []

output_path = '***'

for tile in tiles[:1]:
    try:
        print(f"Processing tile {tile}")

        output_tile_path = os.path.join(output_path, tile)
        if os.path.exists(output_tile_path):
            continue
        else:
            
            # Read bands
            tile_path = os.path.join(input_path, tile)
            arr_bands, metadata = read_sentinel3_bands(tile_path)

            # Read Sun angle
            SunAngle = s3_sun_angle(tile_path)

            # Apply atmospheric correction
            arr_atcor = AtcorSentinel3(arr_bands, SunAngle).apply_all_corrections()

            # Export to GeoTIFF
            os.makedirs(output_tile_path)
            export_sentinel3_to_geotiff(arr_atcor, metadata, output_tile_path)
    
    except Exception as e:
        print(f"Error processing tile {tile}: {e}")
        pending.append(tile)

Processing tile S3B_OL_1_EFR____20220328T130444_20220328T130744_20220329T171929_0179_064_138_1800_LN1_O_NT_002.SEN3


/home/dani/github/WqSat_format/.venv/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
